In [1]:
import requests
import yaml
import os
from datetime import datetime as dt
import pandas as pd
import numpy as np
import boto3
from io import StringIO

In [2]:
with open('config.yml', 'r') as file:
    config = yaml.safe_load(file)

api_creds = config['apiserver']
ticker = "AAPL"

In [3]:
# Establish API call parameters
url = api_creds['requesturl']

headers = {"X-RapidAPI-Key": api_creds['apikey'],
              "X-RapidAPI-Host": api_creds['host']}

params = {"function": "TIME_SERIES_DAILY",
                "symbol": ticker,
                "outputsize": "compact",
                "datatype": "json"}

In [4]:
# GET request
resp = requests.get(url, headers=headers, params=params)

In [5]:
resp.status_code

200

In [6]:
resp.headers['content-type']

'application/json'

In [7]:
# convert response to JSON
content = resp.json()

In [8]:
# extract metadata
symbol = content['Meta Data']['2. Symbol']
last_updated = content['Meta Data']['3. Last Refreshed']
print(symbol)
print(last_updated)

AAPL
2023-09-14


In [9]:
# get JSON body content
data = content['Time Series (Daily)']

In [10]:
# convert JSON to dataframe
df = pd.DataFrame(data).T
df.head()

,1. open,2. high,3. low,4. close,5. volume
2023-09-14,173.9700,176.1000,173.5800,175.7400,59639771
2023-09-13,176.5100,177.3000,173.9800,174.2100,84267928
2023-09-12,179.4900,180.1300,174.8200,176.3000,90370192
2023-09-11,180.0700,180.3000,177.3400,179.3600,58953052
2023-09-08,178.3500,180.2390,177.7900,178.1800,65602066


In [11]:
# clean column names
cols = [x.split(' ')[-1] for x in df.columns]
df.columns = cols
df.columns

Index(['open', 'high', 'low', 'close', 'volume'], dtype='object')

In [12]:
# change data to type numeric
for col in df.columns:
    df[col] = pd.to_numeric(df[col])
    
df.head()

,open,high,low,close,volume
2023-09-14,173.97,176.100,173.58,175.74,59639771
2023-09-13,176.51,177.300,173.98,174.21,84267928
2023-09-12,179.49,180.130,174.82,176.30,90370192
2023-09-11,180.07,180.300,177.34,179.36,58953052
2023-09-08,178.35,180.239,177.79,178.18,65602066


In [13]:
# make index with observation date into a new column with datetime format
df = df.reset_index()
df.columns = ['obs_date'] + list(df.columns[1:])
df['obs_date'] = pd.to_datetime(df['obs_date'])
df.head()

,obs_date,open,high,low,close,volume
0,2023-09-14,173.97,176.100,173.58,175.74,59639771
1,2023-09-13,176.51,177.300,173.98,174.21,84267928
2,2023-09-12,179.49,180.130,174.82,176.30,90370192
3,2023-09-11,180.07,180.300,177.34,179.36,58953052
4,2023-09-08,178.35,180.239,177.79,178.18,65602066


In [14]:
df.dtypes

obs_date    datetime64[ns]
open               float64
high               float64
low                float64
close              float64
volume               int64
dtype: object

In [15]:
# copy stock ticker into new column
df = pd.concat([pd.DataFrame(data=[symbol]*len(df), columns=['ticker']), df], axis=1)
df.head()

,ticker,obs_date,open,high,low,close,volume
0,AAPL,2023-09-14,173.97,176.100,173.58,175.74,59639771
1,AAPL,2023-09-13,176.51,177.300,173.98,174.21,84267928
2,AAPL,2023-09-12,179.49,180.130,174.82,176.30,90370192
3,AAPL,2023-09-11,180.07,180.300,177.34,179.36,58953052
4,AAPL,2023-09-08,178.35,180.239,177.79,178.18,65602066


In [16]:
# Establish connection to AWS environment
aws_creds = config['awsaccount']

os.environ["AWS_ACCESS_KEY_ID"] = aws_creds['accesskey']
os.environ["AWS_SECRET_ACCESS_KEY"] = aws_creds['secretkey']
os.environ["AWS_REGION"] = "us-east-1"

s3 = boto3.resource('s3')

In [17]:
bucket = "stocks-daily-ohlc"

In [18]:
buffer = StringIO()
df.to_csv(buffer, index = False)

In [19]:
s3.Object(bucket, symbol+'.csv').put(Body = buffer.getvalue())

{'ResponseMetadata': {'RequestId': '0GF6JV8T1HG4N4EE',
  'HostId': '7ptKuKWV7Wmm29az+9raXGu6WV7YNoTrMtzzSIYBB+nI+dwpsmLqAzULRlJEQxU2PF+YVYYnTrM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '7ptKuKWV7Wmm29az+9raXGu6WV7YNoTrMtzzSIYBB+nI+dwpsmLqAzULRlJEQxU2PF+YVYYnTrM=',
   'x-amz-request-id': '0GF6JV8T1HG4N4EE',
   'date': 'Thu, 14 Sep 2023 20:34:33 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"09bc7cb21266b175b453100a5dd6c038"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"09bc7cb21266b175b453100a5dd6c038"',
 'ServerSideEncryption': 'AES256'}